## Семинар 1 Индекс

## Intro

### работа с файлами и папками

In [1]:
import os

curr_dir = os.getcwd()
filepath = os.path.join(curr_dir, 'test.txt')

### os.path  
путь до файла

In [2]:
# возвращает полный путь до папки/файла по имени файла / папки
print(os.path.abspath(filepath))


# возвращает имя файла / папки по полному ти до него
print(os.path.basename(filepath))


# проверить существование директории - True / False
print(os.path.exists(curr_dir))

/Users/victoriaregina/infosearch/1 Index/test.txt
test.txt
True


### os.listdir  
возвращает список файлов в данной директории

In [3]:
os.listdir(curr_dir)

['.ipynb_checkpoints', 'lec1_Index.pdf', 'sem1_Index.ipynb', 'test.txt']

При обходе файлов не забывайте исключать системные директории, такие как .DS_Store

### os.walk
root - начальная директория  
dirs - список поддиректорий (папок)   
files - список файлов в этих поддиректориях  

In [4]:
for root, dirs, files in os.walk(curr_dir):
    for name in files:
        print(os.path.join(root, name))

/Users/victoriaregina/infosearch/1 Index/lec1_Index.pdf
/Users/victoriaregina/infosearch/1 Index/sem1_Index.ipynb
/Users/victoriaregina/infosearch/1 Index/test.txt
/Users/victoriaregina/infosearch/1 Index/.ipynb_checkpoints/sem1_Index-checkpoint.ipynb


> __os.walk__ возвращает генератор, это значит, что получить его элементы можно только проитерировавшись по нему  
но его легко можно превратить в list и увидеть все его значения

In [5]:
list(os.walk(curr_dir))

[('/Users/victoriaregina/infosearch/1 Index',
  ['.ipynb_checkpoints'],
  ['lec1_Index.pdf', 'sem1_Index.ipynb', 'test.txt']),
 ('/Users/victoriaregina/infosearch/1 Index/.ipynb_checkpoints',
  [],
  ['sem1_Index-checkpoint.ipynb'])]

### чтение файла 

In [6]:
fpath = 'test.txt'


# одним массивом  
with open(fpath, 'r') as f:  
    text = f.read() 

    
#по строкам, в конце каждой строки \n  
with open(fpath, 'r') as f:   
    text = f.readlines() 

    
#по строкам, без \n   
with open(fpath, 'r') as f:   
    text = f.read().splitlines() 

Напоминание про enumerate:    
> При итерации по списку вы можете помимо самого элемента получить его порядковый номер    
``` for i, element in enumerate(your_list): ...  ```    
Иногда для получения элемента делают так -  ``` your_list[i] ```, не надо так

##  Индекс 

Сам по себе индекс - это просто формат хранения данных, он не может осуществлять поиск. Для этого необходимо добавить к нему определенную метрику. Это может быть что-то простое типа булева поиска, а может быть что-то более специфическое или кастомное под задачу.

Давайте посмотрим, что полезного можно вытащить из самого индекса.    
По сути, индекс - это информация о частоте встречаемости слова в каждом документе.   
Из этого можно понять, например:
1. какое слово является самым часто употребимым / редким
2. какие слова встречаются всегда вместе - так можно парсить твиттер, fb, форумы и отлавливать новые устойчивые выражения в речи
3. как эти документы кластеризуются по N тематикам согласно словам, которые в них упоминаются 

## __Задача__: 

**Data:** Коллекция субтитров сезонов Друзьей. Одна серия - один документ.

**To do:** Постройте небольшой модуль поискового движка, который сможет осуществлять поиск по коллекции документов.
На входе запрос и проиндексированная коллекция (в том виде, как посчитаете нужным), на выходе отсортированный по релевантности с запросом список документов коллекции. 

Релизуйте:
    - функцию препроцессинга данных
    - функцию индексирования данных
    - функцию метрики релевантности 
    - собственно, функцию поиска

[download_friends_corpus](https://yadi.sk/d/yVO1QV98CDibpw)

Напоминание про defaultdict: 
> В качестве multiple values словаря рекомендую использовать ``` collections.defaultdict ```                          
> Так можно избежать конструкции ``` dict.setdefault(key, default=None) ```

In [22]:
import os
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+')

In [23]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

In [24]:
import nltk
import collections
import re

In [25]:
nltk.download("stopwords")
from nltk.corpus import stopwords
russian_stopwords = stopwords.words("russian")

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/victoriaregina/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [26]:
nww = [] #array with names of series and words 
tokens = [] #all the tokens of all docs
j = 0
folders = [f for f in os.listdir('/Users/victoriaregina/Downloads/friends') if 'Store' not in f]
for f in folders:
    for file in os.listdir('/Users/victoriaregina/Downloads/friends/'+ f):
        filepath = '/Users/victoriaregina/Downloads/friends/'+ f + '/'+ file
        with open(filepath, 'r') as doc:
            normal_forms = []
            normal_forms.append(file)
            doc = doc.read()
            tokenizer = RegexpTokenizer(r'\w+')
            for t in tokenizer.tokenize(doc):
                t = morph.parse(t)[0]
                if t.normal_form not in russian_stopwords and t.normal_form != 'это' and not re.match(r'[0-9A-Za-z]', t.normal_form):
                    normal_forms.append(t.normal_form)
                    tokens.append(t.normal_form)
            nww.append(normal_forms)

In [29]:
count = collections.Counter()
for word in tokens:
    count[word] += 1

In [31]:
termdoc = dict.fromkeys(list(count), {})

In [33]:
word_freq = []
for file in nww:
    episode = []
    c = collections.Counter()
    for word in file[1:]:
        c[word] += 1
    episode.append(file[0])
    episode.append(dict(c))
    word_freq.append(episode)

С помощью обратного индекса посчитайте:  


a) какое слово является самым частотным

b) какое самым редким

c) какой набор слов есть во всех документах коллекции

d) какой сезон был самым популярным у Чендлера? у Моники?

e) кто из главных героев статистически самый популярный? 


In [37]:
m = 0
k = 'k'
top = {}
in_all_docs = []
for key, value in termdoc.items():
    for i in range(len(word_freq)):
        try:
            termdoc[key].update(dict([(word_freq[i][0], word_freq[i][1][key])]))
        except KeyError:
            termdoc[key].update(dict([(word_freq[i][0], 0)]))
    all_numb = []
    for name, numb in value.items():
        all_numb.append(numb)
    top[key] = sum(all_numb)
    if sum(all_numb)> m:
        m = sum(all_numb)
        k = key
    if len(value) == 165 and 0 not in value.values():
        in_all_docs.append(key)

# Задание (a)

In [56]:
print("Самое частотное слово:", k, max_key)

Самое частотное слово: весь 3100


# Задание (b)

In [52]:
unfreq = []
for s in sorted(top.items(), key=lambda item: item[1])[:10]:
    unfreq.append(s[0])
print('Самые редкие слова:', ', '.join(unfreq))

Самые редкие слова: придурь, горб, придача, старшеклассник, обмотать, чмо, зацикливаться, истерический, всхлип, лимож


# Задание (c)

In [53]:
in_all_docs
print('Набор слов, который есть во всех документах:', ', '.join(in_all_docs))

Набор слов, который есть во всех документах: весь, просто, мочь, знать, хотеть, сказать, ещё, думать


# Задание (e)

In [55]:
stat = {}
top = []
for i in ['росс', 'фиби', 'моника', 'чендлер', 'джо', 'рэйчел']:
    n = top_sort[i]
    stat[n] = i
for v in stat.keys():
    top.append(v)
print('Самый популярный герой:', stat[max(top)], max(top))

Самый популярный герой: росс 1016
